### Build an LLM from scratch (i.e. ChatGPT model)

**Chapter 1 - Understanding LLMs**

- LLMs 'understand' human language in that they can generate text that appears coherent/ contextually relevant<br>
<br>

- The 'large' in LLM refers to the large number of parameters and size of the dataset the model is trained on
- LLMs have a 'transformer' architecture, which allows them to pay 'selective' attention to parts of an input
- LLMs fall under the category of deep learning, which is a subset of machine learning; a subset of AI
- LLMs are trained on datasets that allow them to classify things (unlike manual rule-setting)<br>
<br>

- LLMs are best used for tasks that involve parsing and generating text in specialised fields like medicine/ law<br>
<br>

- Most modern LLMs are implemented using PyTorch - domain specific ones can outperform general ones like ChatGPT
- Custom LLMs are also smaller scale and can be deployed from laptops/ phones, where biggers ones are more costly
- Creating an LLM involves 2 phases - pre-training (using large datasets) and fine-tuning (using narrower datasets)
- Pre-training uses raw, unlabelled text, and gives the LLM some simple capabilities like text completion
- Fine-tuning can be 'instruction' or 'classification', which uses 'QnA' style data or 'labelled' data respectively<br>
<br>

- The original transformer architecture was developed to translate English into German and French texts
- Transformers consist of:
    - An 'encoder' that processes the input text into numerical representations
    
    - A 'decoder' that processes the numerical representations and generates output text
- LLMs have a 'self-attention' mechanism that allows the model to weight the importance of different parts of an input
- Generative pre-trained transformers (GPT, like ChatGPT) use this mechanism to perform:
    - Zero shot learning, which are tasks that are carried out without any prior examples
    
    - Few shot learning, which involve some examples the user provides as input<br>
<br>

- Pre-trained models of current ChatGPT versions are versatile and good for being fine-tuned for specific purposes<br>
<br>

- GPT models were pre-trained on a next-word prediction task, which predicts the next word in a text based on the previous words
- This training is a form of self-labeling, where the structure of the data itself is the label (i.e. the predicted word)
- GPT architecture actually only contains the 'decoder' part of the transformer, AKA 'autoregressive' models
    - These models incorporate their previous outputs as inputs for future predictions
- GPT models are also interesting in that they can perform tasks that they were not trained for
    - Language translation is an 'emergent' capability of GPT, showing that diverse tasks do not require diverse models<br>
<br>

- Building an LLM first requires data preparation and implementing the architecture (both can be done low-cost)<br>

**Chapter 2 - Working with Text data**

- Pre-training the LLM involves preparing text data by splitting it into individual word and subword 'tokens'
- These are then encoded into vector representations (i.e. lists containing numbers)<br>
<br>

- Text embedding is the process of converting text into numerical vectors (done so as LLMs cannot process raw text)
- Word embeddings can have more than 1 dimension (i.e. more than 1 number in a list), more dimensions = more computation<br>
<br>

- Tokenizing is the process of splitting text into tokens, where each word or punctuation is a single tokens<br>
<br>

The following code reads in a short story, "The Verdict", to be used as text data for the tokenization process

In [ ]:
file_path = "the-verdict.txt"

with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print("First 100 characters: " + str(raw_text[:99]))

Total number of characters: 20479
First 100 characters: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


1. Split the text into singular words and punctuations (i.e. singular tokens)

- Python has a regular expression library that can be used to split text into singular words, as seen in the code below

In [2]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print("Total number of tokens: " + str(len(preprocessed)))
print("First 10 individual words: " + str(preprocessed[:10]))


Total number of tokens: 4690
First 10 individual words: ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius']


2. Convert the tokens (retrieved words and punctuations) into token IDs

- The tokens are stored in an alphabetical vocabulary, where each token has a unique ID

In [3]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print("Total vocab size: " + str(vocab_size))

vocab = {token:integer for integer, token in enumerate(all_words)}

print("Tokens 21 - 25 of the vocab:")
for i, item in enumerate(vocab.items()):
    if i > 20 and i <= 25:
        print(item)

Total vocab size: 1130
Tokens 21 - 25 of the vocab:
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)


3. Apply the vocabulary to convert new text data into tokens

- This models the encode and decode processes of a transformer, which can be carried out by a tokenizer class

- However, this will only be able to tokenize text that is within the vocabulary (duh)

In [13]:
class Tokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\\])', r'\1', text)

        return text
    
tokenizer = Tokenizer(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

print("Token ids of the words in the text: " + str(ids))
print("Decoded result of the tokenised words: " + str(tokenizer.decode(ids)))

Token ids of the words in the text: [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
Decoded result of the tokenised words: " It ' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


4. Modify the tokeniser to handle unknown words

- Tokens can be added to the vocabulary to represent unknown words and text separations

- The encode function of the tokeniser class can be modified to tokenise these special cases in the text data

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print("New amount of tokens in vocab: " + str(len(vocab.items())))

def new_encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int
                    else "<|unk|>" for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

Tokenizer.encode = new_encode

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

print("Joined sample text with separation: " + text)

new_tokeniser = Tokenizer(vocab)

print("Decoded encoded text: " + tokenizer.decode(tokenizer.encode(text)))

New amount of tokens in vocab: 1132
Joined sample text with separation: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
Decoded encoded text: <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
